# Import dependencies

In [1]:
import pandas as pd
import pickle
# %pip install pyarrow

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")

## Q1: Read the data for January. How many columns are there?

In [4]:
df_train.shape

(3066766, 19)